<a href="https://colab.research.google.com/github/elluis1001/Project-4/blob/main/Nidhi/Diabetes_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PART 1 : Install Dependencies & Run Spark Session

In [ ]:
#install pyspark
! pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=510024ea95b62b8a225e84ff33aea1f6c2b5b8b6110dd4c9d6776a3acb29102c
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
#create a sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark").getOrCreate()

# PART 2: Clone & Explore dataset

In [44]:
#clone the diabetes dataset from the github repository
# if you get error saying "Project-4 directory already exists, move to next cell"
! git clone https://github.com/elluis1001/Project-4/

fatal: destination path 'Project-4' already exists and is not an empty directory.


In [45]:
#check if the dataset exists
! ls /content/Project-4/Nidhi/dataset/

diabetes.csv  new_test.csv


In [ ]:
#create spark dataframe
df = spark.read.csv("/content/Project-4/Nidhi/dataset/diabetes.csv", header=True, inferSchema=True)

In [ ]:
#display the dataframe
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [ ]:
#print the schema
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [ ]:
#count the total no. of diabetic and non-diabetic class
print((df.count(), len(df.columns)))
df.groupBy('Outcome').count().show()

(768, 9)
+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  268|
|      0|  500|
+-------+-----+



In [ ]:
#get the summary statistics
df.describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------------+------------------+------------------+
|summary|       Pregnancies|          Glucose|     BloodPressure|     SkinThickness|           Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------------+------------------+------------------+
|  count|               768|              768|               768|               768|               768|               768|                     768|               768|               768|
|   mean|3.8450520833333335|     120.89453125|       69.10546875|20.536458333333332| 79.79947916666667|31.992578124999977|      0.4718763020833327|33.240885416666664|0.3489583333333333|
| stddev|  3.36957806269887|31.97261819513622|19.355807170644777|15.95

# PART 3: Data Cleaning & Preparation

In [ ]:
#check for null values
for col in df.columns:
  print(col+":",df[df[col].isNull()].count())

Pregnancies: 0
Glucose: 0
BloodPressure: 0
SkinThickness: 0
Insulin: 0
BMI: 0
DiabetesPedigreeFunction: 0
Age: 0
Outcome: 0


In [ ]:
#look for the unnecessary values present
def count_zeros():
  columns_list = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
  for i in columns_list:
    print(i+":",df[df[i]==0].count())

In [ ]:
count_zeros()

Glucose: 5
BloodPressure: 35
SkinThickness: 227
Insulin: 374
BMI: 11


In [ ]:
#calculate and replace the unnecessary values by the mean value
from pyspark.sql.functions import *

for i in df.columns[1:6]:
  data = df.agg({i:"mean"}).first()[0]
  print("mean value for {} is {}".format(i,int(data)))
  df = df.withColumn(i, when(df[i]==0, int(data)).otherwise(df[i]))

mean value for Glucose is 120
mean value for BloodPressure is 69
mean value for SkinThickness is 20
mean value for Insulin is 79
mean value for BMI is 31


In [ ]:
#display the dataframe
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|     79|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|     79|26.6|                   0.351| 31|      0|
|          8|    183|           64|           20|     79|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|           20|     79|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


# PART 4: Correlation Analysis & Feature Selection

In [ ]:
#find the correlation among the set of input & output variables
for i in df.columns:
  print("Correlation to outcome for {} is {}".format(i, df.stat.corr("Outcome",i)))

Correlation to outcome for Pregnancies is 0.22189815303398638
Correlation to outcome for Glucose is 0.49288410274882094
Correlation to outcome for BloodPressure is 0.16287909949861834
Correlation to outcome for SkinThickness is 0.171856814176564
Correlation to outcome for Insulin is 0.17869558803050842
Correlation to outcome for BMI is 0.31289043493401536
Correlation to outcome for DiabetesPedigreeFunction is 0.17384406565296007
Correlation to outcome for Age is 0.23835598302719757
Correlation to outcome for Outcome is 1.0


In [ ]:
#feature selection
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                                         'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'], outputCol='features')
output_data = assembler.transform(df)

In [ ]:
#print the schema
output_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
#display dataframe
output_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          6|    148|           72|           35|     79|33.6|                   0.627| 50|      1|[6.0,148.0,72.0,3...|
|          1|     85|           66|           29|     79|26.6|                   0.351| 31|      0|[1.0,85.0,66.0,29...|
|          8|    183|           64|           20|     79|23.3|                   0.672| 32|      1|[8.0,183.0,64.0,2...|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|[1.0,89.0,66.0,23...|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|[0.0,137.0,40.0,3...|
|          5|    116|           

# PART 5: Split Dataset & Build the Model

In [ ]:
#create final data
from pyspark.ml.classification import LogisticRegression

final_data = output_data.select('features','Outcome')

In [ ]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Outcome: integer (nullable = true)



In [ ]:
#split the dataset ; build the model
train, test = final_data.randomSplit([0.7, 0.3])
models = LogisticRegression(labelCol= 'Outcome')
model = models.fit(train)

In [ ]:
#summary of the model
summary = model.summary
summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|            Outcome|         prediction|
+-------+-------------------+-------------------+
|  count|                560|                560|
|   mean|0.35714285714285715|0.29464285714285715|
| stddev|0.47958581670432143|0.45628963389776417|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



# PART 6: Evaluate and Save the Model

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictions = model.evaluate(test)

In [ ]:
predictions.predictions.show(20)

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[0.0,57.0,60.0,20...|      0|[3.65824823937824...|[0.97487015831260...|       0.0|
|[0.0,73.0,69.0,20...|      0|[4.56131348669421...|[0.98965971250753...|       0.0|
|[0.0,91.0,68.0,32...|      0|[2.00714157022038...|[0.88154486027674...|       0.0|
|[0.0,93.0,60.0,20...|      0|[2.39155491800957...|[0.91618105273407...|       0.0|
|[0.0,93.0,60.0,25...|      0|[2.86432676913922...|[0.94605454431977...|       0.0|
|[0.0,93.0,100.0,3...|      0|[0.55916366331394...|[0.63625900603525...|       0.0|
|[0.0,98.0,82.0,15...|      0|[3.38678732752321...|[0.96728904515270...|       0.0|
|[0.0,101.0,64.0,1...|      0|[3.78658067906886...|[0.97782967204329...|       0.0|
|[0.0,101.0,65.0,2...|      0|[3.48572715722291...|[0.97027892400908...|    

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol= 'rawPrediction', labelCol='Outcome')
evaluator.evaluate(model.transform(test))

0.8296218487394958

In [ ]:
# save model
model.save("model")

In [ ]:
# load saved model back to the environment
from pyspark.ml.classification import LogisticRegressionModel

model = LogisticRegressionModel.load('model')

# PART 7: Prediction on New Data with the saved model

In [ ]:
#create a new spark dataframe
test_df = spark.read.csv('/content/Project-4/Nidhi/dataset/new_test.csv', header=True, inferSchema=True)

In [ ]:
#print the schema
test_df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)



In [ ]:
#create an additional feature merged column
test_data = assembler.transform(test_df)

In [ ]:
#print the schema
test_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
#use model to make predictions
results = model.transform(test_data)
results.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [ ]:
#display the predictions
results.select('features','prediction').show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[1.0,190.0,78.0,3...|       1.0|
|[0.0,80.0,84.0,36...|       0.0|
|[2.0,138.0,82.0,4...|       1.0|
|[1.0,110.0,63.0,4...|       1.0|
+--------------------+----------+

